# Convert First-Order Network Data
Making InfoMap work on first order networks for comparison.

In [1]:
import signac
import pandas as pd

project = signac.get_project()
job = project.find_jobs({"year": 2011, "quarter": 1}).next()

In [2]:
first_order_network = pd.read_csv(job.fn('all_edges.tsv'), sep='\t',
                                  header=None, names=['source', 'dest', 'weight'])

In [3]:
def first_order_prepare_infomap(job):
    import json
    from collections import OrderedDict
    state_nodes = OrderedDict()
    edges = set()
    first_order_network = pd.read_csv(job.fn('all_edges.tsv'), sep='\t',
                                      header=None, names=['source', 'dest', 'weight'])

    unique_ids = list(map(int, sorted(set(first_order_network['source'].unique()).union(
        set(first_order_network['dest'].unique())))))

    keys = list(range(1, len(unique_ids)+1))
    vertices = OrderedDict({k: v for (k, v) in zip(unique_ids, keys)})

    # Save the mapping to the arbitrary keys used for InfoMap compatibility
    with open(job.fn('first_order_infomap_vertex_keys.json'), 'w') as f:
        json.dump(vertices, f)
    with open(job.fn('first_order_infomap_key_vertices.json'), 'w') as f:
        json.dump({v: k for k, v in vertices.items()}, f)

    with open(job.fn('first_order_infomap.txt'), 'w') as f:
        # Written in "link list format" for InfoMap
        for row in first_order_network.itertuples():
            f.write('{} {} {}\n'.format(vertices[row.source], vertices[row.dest], row.weight))

In [4]:
first_order_prepare_infomap(job)